# Scrape Página Senado de la República 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import itertools

from selenium import webdriver
import time

###  se establece la lista de urls que lleva a la información de cada senador 

In [ ]:
def scrap_url_senadores():
    html = requests.get('http://www.senado.gov.co/el-senado/senadores').text
    sopa = BeautifulSoup(html, 'lxml')
    
    senadores_prefix = 'http://www.senado.gov.co/el-senado/senadores/213-directorio/'
    senadores_sufix = re.findall('directorio/(.+?)\">', str(sopa.find_all('tbody')))
    url_senadores = [senadores_prefix + x for x in senadores_sufix]
    
    return url_senadores

#####  se aplica la función 

In [ ]:
senador_url = scrap_url_senadores() 

### Algoritmo para sacar información de cada senador ( nombre, partido) 

In [ ]:
def senador_info(url):
    try:
        html = requests.get(url).text
        sopa = BeautifulSoup(html, 'lxml')
    
        nombre_senador = re.findall('\t([A-ZÁÉÍÓU].+?)\t',str(sopa.find_all('p', id='contact-name')))[-1]
        departamento_senador = re.findall('\t([A-ZÁÉÍÓU].+?)\t',str(sopa.find_all('p', id='contact-position')))[-1]
        partido_senador = re.findall('>(.+?)</p>',str(sopa.find_all('p', id='contact-country')))[-1]
        
        return{'senador':nombre_senador,'Departamento':departamento_senador,'Partido':partido_senador}
    except:
        nombre_senador = 'link dañado'
        departamento_senador = 'link dañado'
        partido_senador = 'link dañado'
        return{'senador':nombre_senador,'Departamento':departamento_senador,'Partido':partido_senador}

###  se aplica la función y se crea la base de datos 

In [ ]:
senador_Dicc = list(map(senador_info,senador_url))
senador_DF = pd.DataFrame(senador_Dicc)
senador_DF.rename(columns={'senador':'congresista'}, inplace=True)
senado_DF = pd.DataFrame(senador_DF[['congresista','Partido','Camara']])